# データサイエンス100本ノック（構造化データ加工編）

In [ ]:
!pip install --upgrade pip
!pip install -U pandas numpy scikit-learn
!pip install imbalanced-learn

import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

df_customer = pd.read_csv('data/customer.csv')
df_category = pd.read_csv('data/category.csv')
df_product = pd.read_csv('data/product.csv')
df_receipt = pd.read_csv('data/receipt.csv')
df_store = pd.read_csv('data/store.csv')
df_geocode = pd.read_csv('data/geocode.csv')


In [ ]:
df_geocode['latitude'].head()

# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
df_tmp= pd.merge(df_receipt,df_customer, on='customer_id', how='inner')
df_tmp['era']=df_tmp['age'].apply(lambda x: math.floor(x/10)*10)
df_sales_summary= df_tmp.pivot_table(index='era',columns='gender_cd',aggfunc='sum',values='amount').reset_index()
df_sales_summary.columns= ['era','male','female','unknown']
df_sales_summary

---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [ ]:
df_sales_summary2 = df_sales_summary.set_index('era').stack().reset_index() \
    .replace({'male':'00','female':'01','unknown':'99'})
#set_index()は特定の列をindexに。stack()は列を行方向にピボット。
df_sales_summary2.columns = ['era','gender_cd','amount']
df_sales_summary2

---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
print(df_customer['birth_day'].dtypes)
print(type(df_customer['birth_day'][0]))
pd.concat([df_customer['customer_id'],pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')],axis=1).head()

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_customer['customer_id'],pd.to_datetime(df_customer['application_date'])], axis=1).head()


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_ymd'].astype(str))],axis=1).head()

---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s')],axis=1).head()

---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.year],axis=1).head()

---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.strftime('%m')],axis=1).head()

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']], pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%d')],axis=1).head()

---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_tmp = df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_flg']= df_tmp['amount'].apply(lambda x: 0 if x <= 2000 else 1)
df_tmp.head()

---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [ ]:
df_customer['area_flg']=df_customer['postal_cd'].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)
pd.merge(df_receipt, df_customer[['customer_id','area_flg']], on='customer_id', how='inner').groupby('area_flg').agg({'customer_id': 'nunique'})

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [ ]:
df_customer['pref_cd']= df_customer['address'].str[0:3].map({'埼玉県': '11', '千葉県': '12', '東京都': '13', '神奈川': '14'})
df_customer[['customer_id','address','pref_cd']].head()

---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [ ]:
df_tmp=df_receipt[['customer_id','amount']].groupby('customer_id').amount.sum().reset_index()
q1 = df_tmp['amount'].quantile(0.25)
q2 = df_tmp['amount'].quantile(0.5)
q3 = df_tmp['amount'].quantile(0.75)

df_tmp['amount_flg']= df_tmp['amount'].apply(lambda x: 1 if x < q1 else 2 if q1<= x < q2 else 3 if q2<= x < q3 else 4)
df_tmp.head()

In [ ]:
df_tmp_2=df_receipt[['customer_id','amount']].groupby('customer_id').amount.sum().reset_index()
q4 = np.quantile(df_tmp_2['amount'], 0.25)
q5 = np.quantile(df_tmp_2['amount'], 0.5)
q6 = np.quantile(df_tmp_2['amount'], 0.5)

def pct_group(x):
    if x < q4:
        return 1
    elif q4<= x < q5:
        return 2
    elif q5<= x < q6:
        return 3
    else:
        return 4

df_tmp_2['amount_flg']= df_tmp_2['amount'].apply(lambda x: pct_group(x))
df_tmp_2.head()

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [ ]:
df_customer['era']=pd.cut(df_customer['age'], bins=[0,10,20,30,40,50,60,np.inf], right=False)
df_customer[['customer_id','birth_day','era']].head()

In [ ]:
df_customer['era']=df_customer['age'].apply(lambda x: min(math.floor(x/10)*10, 60))
df_customer[['customer_id','birth_day','era']].head()

In [ ]:
df_customer['era']=df_customer['age'].apply(lambda x: min(math.floor(x/10)*10, 60))
df_customer[['customer_id','birth_day','era']].head()

---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [ ]:
df_tmp = df_customer[['customer_id','birth_day','era']]
df_tmp['era_gender']= df_customer['era'].astype(str) + '代'+ df_customer['gender']
df_tmp['era_gendercd']= df_customer['era'].astype(str) + '_'+ df_customer['gender_cd'].astype(str)
df_tmp.head()

---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [ ]:
pd.get_dummies(df_customer[['customer_id','gender_cd']], columns=['gender_cd']).head()

---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp= df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_ss'] = preprocessing.scale(df_tmp['amount'])
df_tmp.head()

---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp=df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_mm']=preprocessing.minmax_scale(df_tmp['amount'])
df_tmp.describe()

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp=df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_log10']=[math.log10(x) for x in df_tmp['amount']]
df_tmp.head()

---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp= df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_loge']= [math.log(x,math.e) for x in df_tmp['amount']]
df_tmp.head()

---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [ ]:
df_product['unit_profit']=df_product['unit_price']-df_product['unit_cost']
df_product.head()

---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [ ]:
(1 - df_product['unit_cost']/df_product['unit_price']).mean(skipna=True)

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp['new_price']= (df_tmp['unit_cost']/ 0.7).apply(lambda x : np.floor(x)) #math.floorは欠損値エラー起こる
df_tmp['profit_rate']= (1 - df_tmp['unit_cost']/df_tmp['new_price']).head()
df_tmp.head()

---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp['new_price']= (df_tmp['unit_cost']/ 0.7).round()
df_tmp['profit_rate']= (1 - df_tmp['unit_cost']/df_tmp['new_price']).head()
df_tmp.head()

---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp=df_product.copy()
df_tmp['new_price']= df_tmp['unit_cost'].apply(lambda x : np.ceil(x/0.7)) #math.ceil()は欠損値エラー起きる
df_tmp['profit_rate']= (1 - df_tmp['unit_cost']/df_tmp['new_price']).head()
df_tmp.head()

---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp= df_product.copy()
df_tmp['price_taxinc']= df_tmp['unit_price'].apply(lambda x: np.floor(x*1.1))
df_tmp.head()

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [ ]:
df_tmp= pd.merge(df_receipt,df_product)
df_tmp_a= df_tmp.groupby('customer_id').amount.sum().reset_index()
df_tmp_b= df_tmp[df_tmp['category_major_cd']==7].groupby('customer_id').amount.sum().reset_index()
df_tmp= pd.merge(df_tmp_a,df_tmp_b, on='customer_id', how='right')
df_tmp.assign(y_rate= df_tmp['amount_y']/df_tmp['amount_x']).head()

---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
df_tmp= pd.merge(df_receipt[['sales_ymd','customer_id']],df_customer[['customer_id','application_date']], on='customer_id',how='inner')

df_tmp['sales_ymd']= pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['application_date']= pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['days']= df_tmp['sales_ymd'] - df_tmp['application_date']

df_tmp.head()

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
df_tmp=pd.merge(df_receipt[['sales_ymd','customer_id']], df_customer[['customer_id','application_date']], on='customer_id', how='inner')

df_tmp['sales_ymd']= pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['application_date']= pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['months'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x: relativedelta(x[0], x[1]).years * 12 + relativedelta(x[0], x[1]).months, axis=1)

df_tmp.head()

---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
df_tmp=pd.merge(df_receipt[['sales_ymd','customer_id']], df_customer[['customer_id','application_date']], on='customer_id', how='inner')

df_tmp['sales_ymd']= pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['application_date']= pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['years'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x: relativedelta(x[0], x[1]).years, axis=1)

df_tmp.head()

---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
df_tmp=pd.merge(df_receipt[['sales_ymd','customer_id']], df_customer[['customer_id','application_date']], on='customer_id', how='inner')
df_tmp['sales_ymd']= pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date']= pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['elapsed_date']= (df_tmp['sales_ymd'] - df_tmp['application_date']).dt.total_seconds()

#解答例
#df_tmp['elapsed_date']= (df_tmp['sales_ymd'].astype('int')/10**9) - (df_tmp['application_date'].astype('int')/10**9)

df_tmp.head()

---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
df_tmp = df_receipt[['customer_id','sales_ymd']]
df_tmp['sales_ymd']= pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['monday']= df_tmp['sales_ymd'].apply(lambda x: x - relativedelta(days=x.weekday()))
df_tmp['from_monday']= df_tmp[['sales_ymd','monday']].apply(lambda x : relativedelta(x[0],x[1]).days, axis=1)
df_tmp

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [ ]:
df_customer.sample(frac=0.01, random_state=0).head()

---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [ ]:
_, df_tmp = train_test_split(df_customer, test_size=0.1, stratify=df_customer['gender_cd'])
df_tmp.groupby('gender_cd').agg({'customer_id':'count'})
#df_tmp.groupby('gender_cd').customer_id.count().reset_index()

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [ ]:
# 3σ：標準偏差σの3個分 ⇒ グループ化後の'amount.sum()'を標準化して比較
df_tmp= df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_tmp['amount_ss']= preprocessing.scale(df_tmp['amount'])
df_tmp.query('abs(amount_ss) >= 3').head()

---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [ ]:
df_tmp= df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').agg({'amount':'sum'})

q1= df_tmp['amount'].quantile(0.25)
q3= df_tmp['amount'].quantile(0.75)
iqr= q3 - q1

a= q1 - (iqr * 1.5)
b= q3 + (iqr * 1.5)

df_tmp.query('not @a <= amount <= @b').head()

---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
df_product.isnull().sum()

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [ ]:
df_product1 = df_product.copy()
print(len(df_product1))
df_product1.dropna(inplace=True)
print(len(df_product1))

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_product2= df_product.fillna({'unit_price':round(np.nanmean(df_product['unit_price'])),\
                                'unit_cost':round(np.nanmean(df_product['unit_cost']))})
df_product2.isnull().sum()

In [ ]:
df_product2 = df_product.copy()
up_mean= round(df_product2.unit_price.mean())
uc_mean= round(df_product2.unit_cost.mean())
df_product2= df_product2.fillna({'unit_price':up_mean,'unit_cost':uc_mean})
df_product2.isnull().sum()

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_product3= df_product.fillna({'unit_price': round(np.nanmedian(df_product['unit_price'])), 'unit_cost': round(np.nanmedian(df_product['unit_cost']))})
df_product3.isnull().sum()

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_tmp= df_product.groupby('category_small_cd').agg({'unit_price':'median','unit_cost':'median'}).reset_index()
df_tmp.columns= ['category_small_cd','price_median','cost_median']
df_product4= pd.merge(df_product,df_tmp, on='category_small_cd', how='inner')

df_product4['unit_price']= df_product4[['unit_price','price_median']].apply(lambda x: round(x[1]) if np.isnan(x[0]) else x[0], axis=1)
df_product4['unit_cost']= df_product4[['unit_cost','cost_median']].apply(lambda x: round(x[1]) if np.isnan(x[0]) else x[0], axis=1)

df_product4.isnull().sum()

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [ ]:
df_tmp= pd.merge(df_customer['customer_id'],df_receipt[['customer_id','sales_ymd','amount']],on='customer_id',how='left').fillna(0)
df_tmp1= df_tmp.groupby('customer_id').amount.sum().reset_index()
df_tmp2= df_tmp[df_tmp['sales_ymd'].astype('str').str.startswith("2019")].groupby('customer_id').amount.sum().reset_index()
df_tmp1.columns = ['customer_id','amount_all']
df_tmp2= df_tmp2.rename(columns= {'amount':'amount_2019'})
df_amount= pd.merge(df_tmp1,df_tmp2, on='customer_id', how='left').fillna(0)
df_amount['rate_2019']= (df_amount['amount_2019'] / df_amount['amount_all']).fillna(0)

print(df_amount.isnull().sum())
df_amount[df_amount['rate_2019'] > 0].head()

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [ ]:
#postal_cdに対して複数の緯度経度がある場合がある（1丁目と2丁目とか）
print([df_geocode['postal_cd'].nunique(),len(df_geocode['postal_cd'])]) #確認用

df_customer_1 = pd.merge(df_customer[['customer_id','postal_cd']],df_geocode[['postal_cd','longitude','latitude']], on='postal_cd', how='inner')

#postal_cdに対して複数の緯度経度がある場合、inner結合でもcustomer_idはその分複製される。
print([df_customer_1['customer_id'].nunique(),len(df_customer_1['customer_id'])]) #確認用

df_customer_1 = df_customer_1.groupby('customer_id').agg({'longitude':'mean','latitude':'mean'}).reset_index().rename(columns={'longitude':'longitude_mean','latitude':'latitude_mean'})

print(len(df_customer_1['customer_id'])) #確認用
df_customer_1= pd.merge(df_customer,df_customer_1, on='customer_id', how='inner')
df_customer_1.head(2)

---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [ ]:
#地点間の距離を求める関数を設定
def calc_distance(x1, y1, x2, y2):
    distance = 6371 * math.acos(math.sin(math.radians(y1)) * math.sin(math.radians(y2)) \
                                + math.cos(math.radians(y1)) * math.cos(math.radians(y2)) \
                                * math.cos(math.radians(x1) - math.radians(x2)))
    return distance

df_customer_2 = pd.merge(df_customer_1,df_store, left_on='application_store_cd',right_on='store_cd', how='inner')

df_customer_2['distance']= df_customer_2[['longitude_mean','latitude_mean','longitude','latitude']].apply(lambda x: calc_distance(x[0],x[1],x[2],x[3]), axis=1)

df_customer_2[['customer_id','address_x','address_y','distance']].head()

---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [ ]:
df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_customer_u = pd.merge(df_customer,df_tmp,on='customer_id',how='left').sort_values(['amount','customer_id'],ascending=[False, True])
#複数列のsort_valuesは、リストの後ろ→前の順にソートされていく = 結果、amount最優先で、同一値はcustomer_id順
df_customer_u.drop_duplicates(subset=['customer_name','postal_cd'], keep='first', inplace=True)
print([len(df_customer),len(df_customer_u)])

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [ ]:
df_customer_n = pd.merge(df_customer,df_customer_u[['customer_id','customer_name','postal_cd']], on=['customer_name','postal_cd'], how='inner').rename(columns={'customer_id_x':'customer_id','customer_id_y':'integration_id'})
print([df_customer_n['customer_id'].nunique(),df_customer_n['integration_id'].nunique()])

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

In [ ]:
del df_customer_1
del df_customer_n

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [ ]:
df_tmp = pd.merge(df_customer,df_receipt, on='customer_id', how='inner')
df_train, df_test = train_test_split(df_tmp, test_size = 0.2, random_state=1)
len(df_train)/len(df_tmp)

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [ ]:
df_tmp= df_receipt.copy()
df_tmp['sales_ym']= df_tmp['sales_ymd'].astype('str').str[0:6]
df_tmp= df_tmp.groupby('sales_ym').amount.sum().reset_index()

def split_data(df, train_size, test_size, slide_window, start_point):
    train_start = start_point * slide_window
    test_start = train_start + train_size
    return df[train_start : test_start], df[test_start : test_start + test_size]

train_1, test_1 = split_data(df=df_tmp, train_size=12, test_size=6, slide_window=6, start_point=0)
train_2, test_2 = split_data(df_tmp, 12, 6, 6, 1)
train_3, test_3 = split_data(df_tmp, 12, 6, 6, 2)

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [ ]:
df_tmp= df_receipt.groupby('customer_id').amount.sum().reset_index()

df_tmp= pd.merge(df_customer, df_tmp, on='customer_id', how='left')

df_tmp['sales_flg']= df_tmp['amount'].apply(lambda x: 0 if np.isnan(x) else 1)

print('0の件数: ',len(df_tmp[df_tmp['sales_flg']==0]))
print('1の件数: ',len(df_tmp[df_tmp['sales_flg']==1]))

positive_count= len(df_tmp[df_tmp['sales_flg']==1])
rs= RandomUnderSampler(random_state=1)

df_sample, _ = rs.fit_sample(df_tmp, df_tmp.sales_flg)

print('0の件数: ',len(df_sample[df_sample['sales_flg']==0]))
print('1の件数: ',len(df_sample.query('sales_flg == 1')))

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [ ]:
df_gender=df_customer[['gender_cd','gender']].drop_duplicates().sort_values('gender_cd', ascending=True).reset_index(drop=True)

df_customer_new= df_customer.drop(columns='gender')

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [ ]:
df_tmp= pd.merge(df_product, df_category, how='inner')
df_tmp.head(2)

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
os.makedirs('./data')

df_tmp.to_csv('./data/df_tmp.csv', encoding='utf-8', header=True, index=False, sep=',')

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [ ]:
df_tmp.to_csv('data/df_tmp_2.csv', index=False, encoding='cp932')

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [ ]:
df_tmp.to_csv('data/df_tmp_3.csv', header=False)

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_tmp4= pd.read_csv('./data/df_tmp.csv')
df_tmp4.head()

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [ ]:
df_tmp5= pd.read_csv('./data/df_tmp_3.csv',header = None)
df_tmp5.head()

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_tmp.to_csv('data/df_tmp6.tsv', index=False, sep='\t')

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_tmp6=pd.read_table('data/df_tmp6.tsv')
df_tmp6.head()

# これで１００本終わりです。おつかれさまでした！